In [ ]:
import sys
sys.path.insert(0, "/projects/academic/smuldoon/bengieru/simplicial_model")

import matplotlib.pyplot as plt
import numpy as np
from Neuronal_Cascades_base import neuron, Geometric_Brain_Network
import os
import pickle

In [ ]:
size = 500
GD = 10
nGD = 2
topology = 'Ring'

TIME = 180
trials = 1
C = 10000
Ks = np.linspace(0,1,21)

memory = TIME
rest = 0
thresholds = np.linspace(0,0.6,31)

base_path = '/projects/academic/smuldoon/bengieru/simplicial_model/Model_Variations/line_segment_model/n=%d/GD_vs_nGD=%d_vs_%d/'%(size,GD,nGD)

os.makedirs(base_path, exist_ok = True)

In [ ]:
delta_matrix_min = np.zeros((len(Ks),len(thresholds)))

GB = Geometric_Brain_Network.Geometric_Brain_Network(size, geometric_degree = GD, nongeometric_degree = nGD, manifold = topology)

for j,threshold in enumerate(thresholds):
    distance_matrices = []
    comm_sizes = []
    
    path = base_path + 'threshold=%.2f/'%threshold
    os.makedirs(path, exist_ok = True)
    
    neurons= [neuron.neuron(k, memory = memory, rest = rest, threshold = threshold) for k in range(size)]
    GB.get_neurons(neurons)
    
    for i, K in enumerate(Ks):
        D, Q = GB.make_distance_matrix(TIME, C, trials, K)
    
        distance_matrices.append(D)
        comm_sizes.append(Q)
    
        delta_min, delta_max = GB.compute_persistence(distance_matrices[i], spy = True)
        plt.savefig(path + 'pers_diag_K=%.2f'%(K) + '.pdf')
        
        delta_matrix_min[i][j] = delta_min

    GB.display_comm_sizes(comm_sizes, list(Ks), TIME, C, trials, memory, rest, threshold)
    plt.savefig(path + 'contagion_size_thresh=%.2f'%(threshold) + '.pdf')

    with open(path +  "comm_sizes_thresh=%.2f.pkl"%threshold, "wb") as fp:
        pickle.dump(comm_sizes, fp)
        
with open(base_path + "delta_matrix_mins_%d.pkl"%nGD, "wb") as fp:
    pickle.dump(delta_matrix_min, fp)

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (20,10))
im = ax.imshow(delta_matrix_min, 
               origin = 'lower', 
               interpolation='nearest', 
               aspect='auto',
               cmap = 'copper', 
               extent = [-.05, 1.05, thresholds[0]-0.005, thresholds[-1]+0.005])
fig.suptitle('Ring Topology on %d nodes. GD = %d, nGD = %d,  T = %d, Trials = %d, C = %d, FULL MEMORY, RESTLESS'%(size, GD, nGD, TIME, trials, C), fontsize = 25)
ax.set_title('Threhsolds vs Ks', fontsize = 20)
ax.set_xticks(Ks)
ax.set_yticks(thresholds)
ax.set_xlabel('Ks', fontsize = 15 )
ax.set_ylabel('Thresholds', fontsize = 15)
ax.tick_params(axis = 'both', labelsize = 12)
fig.colorbar(im)
plt.savefig(base_path + 'bifurcation_diag_min.pdf')